In [1]:
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd

In [2]:
stardf = pd.read_csv("clean_stars.csv")
stardf

,Vmag,Plx,e_Plx,B-V,SpType
0,9.10,3.54,1.39,0.482,F5
1,9.27,21.90,3.10,0.999,K3V
2,6.61,2.81,0.63,-0.019,B9
3,8.06,7.75,0.97,0.370,F0V
4,8.55,2.87,1.11,0.902,G8III
...,...,...,...,...,...
9674,8.45,-0.93,1.13,1.404,K5
9675,7.84,4.26,1.00,1.140,K1IIICN...
9676,9.38,3.61,1.36,0.507,G0
9677,7.64,4.75,0.97,0.075,A2


# Finding Absolute Magnitude 

### I copied this equation off the internet ;D 
Apparently we need it and it's a problem is plx is zero because then we get infinity 

In [3]:
stardf=stardf[stardf.Plx !=0]
stardf

,Vmag,Plx,e_Plx,B-V,SpType
0,9.10,3.54,1.39,0.482,F5
1,9.27,21.90,3.10,0.999,K3V
2,6.61,2.81,0.63,-0.019,B9
3,8.06,7.75,0.97,0.370,F0V
4,8.55,2.87,1.11,0.902,G8III
...,...,...,...,...,...
9674,8.45,-0.93,1.13,1.404,K5
9675,7.84,4.26,1.00,1.140,K1IIICN...
9676,9.38,3.61,1.36,0.507,G0
9677,7.64,4.75,0.97,0.075,A2


In [4]:
stardf = stardf.reset_index(drop=True)

In [5]:
stardf

,Vmag,Plx,e_Plx,B-V,SpType
0,9.10,3.54,1.39,0.482,F5
1,9.27,21.90,3.10,0.999,K3V
2,6.61,2.81,0.63,-0.019,B9
3,8.06,7.75,0.97,0.370,F0V
4,8.55,2.87,1.11,0.902,G8III
...,...,...,...,...,...
9672,8.45,-0.93,1.13,1.404,K5
9673,7.84,4.26,1.00,1.140,K1IIICN...
9674,9.38,3.61,1.36,0.507,G0
9675,7.64,4.75,0.97,0.075,A2


In [6]:
stardf["Amag"] = stardf["Vmag"] + 5* (np.log10(abs(stardf["Plx"]))+1)

In [7]:
stardf

,Vmag,Plx,e_Plx,B-V,SpType,Amag
0,9.10,3.54,1.39,0.482,F5,16.845016
1,9.27,21.90,3.10,0.999,K3V,20.972221
2,6.61,2.81,0.63,-0.019,B9,13.853532
3,8.06,7.75,0.97,0.370,F0V,17.506509
4,8.55,2.87,1.11,0.902,G8III,15.839409
...,...,...,...,...,...,...
9672,8.45,-0.93,1.13,1.404,K5,13.292415
9673,7.84,4.26,1.00,1.140,K1IIICN...,15.987048
9674,9.38,3.61,1.36,0.507,G0,17.167536
9675,7.64,4.75,0.97,0.075,A2,16.023468


In [8]:
stardf.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9677 entries, 0 to 9676
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Vmag    9677 non-null   float64
 1   Plx     9677 non-null   float64
 2   e_Plx   9677 non-null   float64
 3   B-V     9677 non-null   float64
 4   SpType  9677 non-null   object 
 5   Amag    9677 non-null   float64
dtypes: float64(5), object(1)
memory usage: 453.7+ KB


In [9]:
#issa FLOAT! woooo

In [10]:
scaler=StandardScaler ()

In [11]:
# oh my god you can't SCALE AN OBJECT .. what happens if I just drop that column 
#scaled_my= scaler.fit_transform(stardf[["Vmag", "Plx", "e_Plx", "SpType", "Amag"]])
#idea now is to convert sptype into a class column and then remove and use the same principles we did in unsupervised hw, THEN!!
# WE USE THOSE CLASSES AS A LABEL FOR THE UNSUPERVISED MACHINE LEARNING 
# you can't tell me I'm not a genius 

# absolute experimental insanity section 

Dwarfs (I, II, III, VII)
Giants (IV, V, VI)
Other Special Stars (None)
So we can sort into 3 categories but then use one of those fancy Hertzsprung Russel Diagrams to reverse engineer the labels for other special stars 

In [12]:
stardf['TargetClass'] = stardf['SpType']

In [13]:
for i in range(len(stardf['TargetClass'])):
    if "V" in stardf.loc[i,'TargetClass']: 
        if "VII" in stardf.loc[i,'TargetClass']: 
            stardf.loc[i,'TargetClass'] = 0 # VII is Dwarf
        else:
            stardf.loc[i,'TargetClass'] = 1 # IV, V, VI are Giants
    elif "I" in stardf.loc[i,'TargetClass']: 
        stardf.loc[i,'TargetClass'] = 0 # I, II, III are Dwarfs
    else: 
        stardf.loc[i,'TargetClass'] = 9 # None
        
stardf['TargetClass']      

0       9
1       1
2       9
3       1
4       0
       ..
9672    9
9673    0
9674    9
9675    9
9676    1
Name: TargetClass, Length: 9677, dtype: object

In [14]:
stardf['TargetClass'].value_counts ()

9    5031
1    2825
0    1821
Name: TargetClass, dtype: int64

In [15]:
stardf

,Vmag,Plx,e_Plx,B-V,SpType,Amag,TargetClass
0,9.10,3.54,1.39,0.482,F5,16.845016,9
1,9.27,21.90,3.10,0.999,K3V,20.972221,1
2,6.61,2.81,0.63,-0.019,B9,13.853532,9
3,8.06,7.75,0.97,0.370,F0V,17.506509,1
4,8.55,2.87,1.11,0.902,G8III,15.839409,0
...,...,...,...,...,...,...,...
9672,8.45,-0.93,1.13,1.404,K5,13.292415,9
9673,7.84,4.26,1.00,1.140,K1IIICN...,15.987048,0
9674,9.38,3.61,1.36,0.507,G0,17.167536,9
9675,7.64,4.75,0.97,0.075,A2,16.023468,9


In [16]:
label=stardf['TargetClass']

In [17]:
stardf= stardf.drop('SpType', axis=1)
stardf

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass
0,9.10,3.54,1.39,0.482,16.845016,9
1,9.27,21.90,3.10,0.999,20.972221,1
2,6.61,2.81,0.63,-0.019,13.853532,9
3,8.06,7.75,0.97,0.370,17.506509,1
4,8.55,2.87,1.11,0.902,15.839409,0
...,...,...,...,...,...,...
9672,8.45,-0.93,1.13,1.404,13.292415,9
9673,7.84,4.26,1.00,1.140,15.987048,0
9674,9.38,3.61,1.36,0.507,17.167536,9
9675,7.64,4.75,0.97,0.075,16.023468,9


In [18]:
scaled_star= scaler.fit_transform(stardf[["Vmag", "Plx", "e_Plx","B-V", "Amag"]])
scaled_star[0]

array([ 0.55261351, -0.36283388,  0.07839639, -0.57365481,  0.04634903])

In [19]:
scaled_stardf =  pd.DataFrame(scaled_star,columns= stardf.columns)
scaled_stardf.head()                                      

ValueError: Shape of passed values is (9677, 5), indices imply (9677, 6)

In [ ]:
pca = PCA(n_components=0.9)
starpca = pca.fit_transform(scaled_stardf)
pca.explained_variance_ratio_

In [ ]:
starpca.shape 

In [ ]:
tsne = TSNE(learning_rate=300,perplexity=40,n_components=3)
tsne_features = tsne.fit_transform(starpca)

In [ ]:
# look if we are going to sort into giant and dwarf then we have predetermined set outcomes so we would need to classify that
#and then teach the program, but if we remove that restriction we can sort the dataset into different stellar classes that
#aren't just dwarf and giant?
# I'm testing this now 

In [ ]:
tsne_features.shape

In [ ]:
plt.subplots (figsize=(10, 8))
plt.scatter(tsne_features[:, 0], tsne_features[:, 1], c=label)

plt.show()

In [ ]:
inertia = []
k = range(1, 11)

In [ ]:
for i in k: 
    km = KMeans(n_clusters=i, random_state=21)
    km.fit(maybestardf)
    inertia.append(km.inertia_)

elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

In [ ]:
plt.subplots (figsize=(10, 8))
plt.grid ()
plt.plot(df_elbow['k'], df_elbow['inertia'])
plt.xticks(range(1,11))
plt.xlabel("Clusters")
plt.ylabel("Inertia")
plt.show()

In [ ]:
pca_df = pd.DataFrame(tsne_features, columns=["pc1", "pc2","pc3"])
pca_df.head()

In [ ]:
km_model = KMeans(n_clusters=3, random_state=21)
km_model.fit(pca_df)

predicted_clusters = km_model.predict(pca_df)
pca_df['group'] = km_model.labels_

pca_df.head()

In [ ]:
import plotly.express as px
fig = px.scatter_3d(
    pca_df,
    x="pc3",
    y="pc2",
    z="pc1",
    color="group",
    symbol="group",
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [ ]:
# I mean the experiment worked, it's fixed the data into 3 very separate clusters, there isn't much overlap between the models 